In [1]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
import os



In [2]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2')

In [3]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# # Split data into training and validation sets
df_train, df_val = train_test_split(df, test_size=.2)

In [4]:
datagen=ImageDataGenerator(rescale=1./255, rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=(0.75, 1),
        shear_range=0.1,
        zoom_range=[0.75, 1],
        horizontal_flip=True,
        validation_split=0.2)


In [5]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="File", y_col="angle", 
                                            class_mode="raw", target_size=(480,640), batch_size=32)

Found 4491 validated image filenames.


In [6]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="File", y_col="angle", 
                                            class_mode="raw", target_size=(480,640), batch_size=32)

Found 1123 validated image filenames.


In [7]:
input_shape = (480, 640, 3)

In [8]:
from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import pooling


In [9]:
# define PilotNet model, with batch normalization included.
def get_model(input_shape):
    model2 = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    return model2

model2 = get_model(input_shape)
#sgd = SGD(learning_rate=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model2.compile(optimizer='Adam', loss="mse") 
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 238, 318, 24)      1824      
_________________________________________________________________
batch_normalization (BatchNo (None, 238, 318, 24)      952       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 117, 157, 36)      21636     
_________________________________________________________________
batch_normalization_1 (Batch (None, 117, 157, 36)      468       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 77, 48)        43248     
_________________________________________________________________
batch_normalization_2 (Batch (None, 57, 77, 48)        228       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 55, 75, 64)        2

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [31]:
STEP_SIZE_TRAIN

140

In [11]:
model_name = 'regression_udacity1'

In [34]:
callback1 = EarlyStopping(monitor="val_loss",
        min_delta=1,  # model should improve by at least 1%
         # amount of epochs  with improvements worse than 1% until the model stops
        verbose=2,
        mode="min",
        restore_best_weights=True) # restore the best model with the lowest validation error)

In [35]:
callback2 = ModelCheckpoint( "./models/" + model_name + ".h5",
        monitor="val_loss",
        verbose=0,
        save_best_only=True,  # save the best model
        mode="min",
        save_freq="epoch") # save every epoch
    

In [36]:
model2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4, callbacks=[callback1, callback2])

Epoch 1/4
140/140 [==============================] - 1231s 9s/step - loss: 0.0440 - val_loss: 0.0461
Restoring model weights from the end of the best epoch.


In [16]:
model2.save_weights('udacity_regression1_loss0.04_weights.h5')

In [18]:
model2.save('udacity_regression1_loss0.04.h5')

In [25]:
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
import numpy as np



In [37]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img((img_path), target_size=(480,640))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) 

In [49]:
def transform_img_fn1(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img((img_path), target_size=(480,640))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
#         x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) 

In [50]:
images = transform_img_fn1([os.path.join(r'C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2/','1479425716180631063.jpg')])


In [51]:
model2.predict(images)

array([[61.508663]], dtype=float32)